In [15]:
import sys
import os
import csv
import copy
import json
import math
import random
import itertools
import numpy as np
import networkx as nx
import tqdm as tqdm
import matplotlib.pyplot as plt
from pathlib import Path
from protocols import MPC_protocol, MPG_protocol, SP_protocol
from graph import network, set_p_edge

from joblib import Parallel, delayed
from networkx.algorithms.community import greedy_modularity_communities
from networkx.drawing.layout import *

# 获取当前 Notebook 的绝对路径
notebook_path = os.path.abspath("")

from config import DATA_PATHS




In [16]:
mkr = ['x','+','d','o','1','2']+['x']*100
dashs = ['-.','--',':','-']+['-']*100
cols = ['gray','g','b','orange','r','k','purple']+['k']*100
linewidth = 2.2
mks = 5.5
fontsize = 14
sys.path.append("..")  # 确保根目录在 Python 路径中
root_path = DATA_PATHS["input_graphs"]

Find the ER for the MPC, MPG, and SP protocols

In [17]:
def load_data(filepath):
    pos = {}
    user = []

    # Step 1: 读取 JSON 文件
    with open(filepath, "r") as f:
        data = json.load(f)

    # Step 2: 初始化图
    G = nx.Graph()

    # Step 3: 添加节点

    for node in data["nodes"]:
        node_id = node["id"]
        x, y = node["latitude"], node["longitude"]
        G.add_node(node_id, location=node["location"], country=node["country"])  # 添加节点到图
        pos[node_id] = (y, x)  # 保存节点位置，注意 (longitude, latitude)

    # Step 4: 添加边
    for edge in data["links"]:
        source = int(edge["source"])
        target = int(edge["target"])
        G.add_edge(source, target, length=edge["length"])  # 添加边到图

    degree_dict = dict(G.degree())
    degree_items = list(degree_dict.items())
    first_node,first_degree = degree_items[0]
    print(f"First node ID: {first_node}, Degree: {first_degree}")

    user.append(data["nodes"][0]["id"])

    return G,user,pos

In [18]:
import networkx as nx
import random
import matplotlib.pyplot as plt
from itertools import product

def multi_iterative_score_partition_with_drawing(
    G, 
    fixed_node, 
    alpha=1.0, 
    beta=1.0,
    max_rounds=10,
    shuffle_nodes=True,
    pos=None,
    output_path=None
):
    """
    多轮迭代版“打分 + 搬家”分社区并绘制图：
      - 社区数 = degree(fixed_node) + 1
      - 初始: 0 号社区放 fixed_node，其余每个邻居一个社区
      - 分配时: score = alpha * distance + beta * community_size
      - 多轮: 如果搬家能让节点的 score 更低, 就搬家, 直到稳定或 max_rounds
      - 绘图可选。

    同时返回：
      - 最终的社区划分 communities
      - 对所有社区(除了 0 号) 进行“从每个社区选一个节点”的所有组合 all_key_nodes_combos，
        并在每个组合前面加上固定节点 fixed_node。
    """
    # ============ 1) 初始化社区容器 ============
    neighbors = list(G.neighbors(fixed_node))
    num_communities = len(neighbors)  # 除 0 号社区外的社区数
    communities = [set() for _ in range(num_communities + 1)]
    visited = set()

    # 第 0 号社区放 fixed_node
    communities[0].add(fixed_node)
    visited.add(fixed_node)

    # 其余邻居各一个社区
    for i, nb in enumerate(neighbors, start=1):
        communities[i].add(nb)
        visited.add(nb)

    # ============ 2) 初次分配剩余节点 ============
    for node in G.nodes():
        if node not in visited:
            best_score = float('inf')
            best_index = None
            # 尝试放进 1~num_communities 各个社区，看 score 哪个最小
            for i, nb in enumerate(neighbors, start=1):
                dist = nx.shortest_path_length(G, source=node, target=nb)
                size = len(communities[i])
                score = alpha * dist + beta * size
                if score < best_score:
                    best_score = score
                    best_index = i
            communities[best_index].add(node)
            visited.add(node)

    # ============ 3) 多轮迭代搬家 ============
    round_num = 0
    while round_num < max_rounds:
        round_num += 1
        moved_count = 0

        # 获取所有节点（除 fixed_node 外）
        all_nodes = [n for n in G.nodes() if n != fixed_node]

        if shuffle_nodes:
            random.shuffle(all_nodes)

        for node in all_nodes:
            # 找到当前所在社区
            current_idx = None
            for i, comm in enumerate(communities):
                if node in comm:
                    current_idx = i
                    break
            
            # 如果在 0 号社区，则说明节点是 fixed_node，不搬家
            if current_idx == 0:
                continue

            # 当前社区分值
            nb_current = neighbors[current_idx - 1]  # 对应邻居
            dist_current = nx.shortest_path_length(G, source=node, target=nb_current)
            size_current = len(communities[current_idx])
            current_score = alpha * dist_current + beta * size_current

            # 尝试搬去别的社区
            best_score = current_score
            best_index = current_idx

            for i, nb in enumerate(neighbors, start=1):
                if i == current_idx:
                    continue
                dist = nx.shortest_path_length(G, source=node, target=nb)
                size = len(communities[i])
                score = alpha * dist + beta * size

                if score < best_score:
                    best_score = score
                    best_index = i

            # 如果找到更好的社区 => 搬家
            if best_index != current_idx:
                communities[current_idx].remove(node)
                communities[best_index].add(node)
                moved_count += 1

        # 如果没人搬家 => 收敛 => 停止
        if moved_count == 0:
            break

    # ============ 4) 计算所有可能的 key_nodes 组合 ============
    # 注意：0 号社区只包含 fixed_node，它是永远固定的，不做枚举
    #       其余社区(1..num_communities)从中各自选 1 个节点，形成一个组合
    all_key_nodes_combos = []

    # 如果有社区是空的，那么不可能从该社区选出节点
    if all(len(communities[i]) > 0 for i in range(1, num_communities + 1)):
        # itertools.product(...) 会返回所有从各社区选1个节点的组合
        # communities[i] 本身是一个 set，不影响 product 的用法
        # 例如 [setA, setB, setC] => product(setA, setB, setC) 会返回 (a, b, c) 等所有组合
        all_products = product(*(communities[i] for i in range(1, num_communities + 1)))

        # 为每个组合都把 fixed_node(第0社区) 放在开头
        for combo in all_products:
            combo_list = [fixed_node] + list(combo)
            all_key_nodes_combos.append(combo_list)
    else:
        # 如果有社区为空，那么没有可行的枚举
        all_key_nodes_combos = []
    # # ============ 5) 绘制社区图 ============
    # if pos is None:
    #     pos = nx.spring_layout(G, seed=42)  # 默认布局

    # colors = ["red", "blue", "green", "orange", "purple", "cyan", "yellow", "pink"]
    
    # plt.figure(figsize=(8, 6))
    
    # # 逐个社区画节点
    # for i, community in enumerate(communities):
    #     nx.draw_networkx_nodes(
    #         G, pos, nodelist=community,
    #         node_color=colors[i % len(colors)], 
    #         label=f"Community {i}",
    #         alpha=0.8,
    #         node_size=100
    #     )

    # # 特别标注固定节点（用红色圆形)
    # nx.draw_networkx_nodes(
    #     G, pos, nodelist=[fixed_node],
    #     node_color="red", node_shape="o",
    #     node_size=100, alpha=0.9, label="Fixed Node"
    # )

    # # 绘制边
    # nx.draw_networkx_edges(G, pos, edge_color="gray", alpha=0.5)
    # # 绘制标签
    # nx.draw_networkx_labels(G, pos, font_size=8, font_color="black")

    # plt.legend(
    #     fontsize=6,  
    #     borderaxespad=0.5, 
    #     labelspacing=0.2,   
    #     loc="upper left",
    #     bbox_to_anchor=(1.05, 1),
    # )
    # plt.title("Graph with Colored Communities (no farthest-node highlight)")
    # plt.tight_layout(pad=2.0)

    # if output_path:
    #     plt.savefig(output_path, dpi=300, bbox_inches='tight')
    #     print(f"Graph saved to {output_path}")
    # else:
    #     plt.show()
    # plt.close()

    # 返回：最终社区划分，以及所有 key_nodes 组合
    return communities, all_key_nodes_combos


In [19]:
def process_single_p(G_original, combo, p, funcs, timesteps, reps):
    """处理单个p值的并行任务函数"""
    # 创建图的深拷贝避免共享状态问题
    G = copy.deepcopy(G_original)
    
    # 设置当前p值的边概率
    set_p_edge(G, p_op=p)
    
    # 为所有协议计算当前p的ER值
    p_ers = []
    for func in funcs:
        er, _, _ = func(G, combo, timesteps=timesteps, reps=reps)
        p_ers.append(er)
    
    return p_ers

In [20]:

def plot_er_vs_p(p_range, ER, funcs, cols, output_path, fontsize=12, figsize=(10, 6), dpi=600):
    """
    绘制 ER 与链接生成概率 p 的关系图，并保存为文件。

    参数:
        p_range (array-like): p 的取值范围。
        ER (list of lists): 每个函数对应的 ER 数据。
        funcs (list): 包含函数的列表，用于生成图例名称。
        cols (list): 每个函数对应的颜色列表。
        output_path (str): 保存图片的路径。
        fontsize (int, 可选): 坐标轴和标签字体大小，默认 12。
        figsize (tuple, 可选): 图形大小，默认 (10, 6)。
        dpi (int, 可选): 图像分辨率，默认 600。
    """
    print(ER)
    nom_list = [str(f).split(' ')[1] for f in funcs]
    plt.figure(figsize =(10,6),dpi=600 )
    plt.grid(linewidth=0.5)
    for i in range(len(funcs)):
        y = plt.plot(p_range, ER[i],
                    color = cols[i],
                    marker = "x",
                    linestyle='None',
                    markersize = 3,
                    #  alpha = 0.5,
                    #  linewidth=linewidth,
                    label = nom_list[i])
    plt.yscale('log')
    plt.legend(fontsize=10)
    plt.tick_params(labelsize=fontsize)

    plt.xlabel('Link generation probability p',fontsize=fontsize)
    plt.ylabel('ER ($\mathregular{GHZ}_5/\ \\mathregular{T_{slot}}$)',fontsize=fontsize)

    ax = plt.gca()
    ax.set_xlim([0.2, 1])
    ax.set_ylim([0.0001, 1])
    # 保存图片
    plt.savefig(output_path, dpi=dpi)
    print(f"Plot saved to {output_path}")

<>:33: SyntaxWarning: invalid escape sequence '\m'
<>:33: SyntaxWarning: invalid escape sequence '\m'
C:\Users\Ao\AppData\Local\Temp\ipykernel_1680\581972065.py:33: SyntaxWarning: invalid escape sequence '\m'
  plt.ylabel('ER ($\mathregular{GHZ}_5/\ \\mathregular{T_{slot}}$)',fontsize=fontsize)


In [21]:
funcs = [MPC_protocol, MPG_protocol, SP_protocol]
p_range = np.linspace(1, 0.2, 50)

timesteps = 100
reps = 200
alpha = 1.4
beta = 0.105
max_rounds = 10
shuffle_nodes = True
root_path = DATA_PATHS["input_graphs"]

# --------------------------
# 设定一个 0 出现次数的阈值
# 只要超过这个数，就判定本次用户组合的该协议 "失败"
zero_threshold = 10
# --------------------------

# 用于存储每个子文件夹中每个协议的 SR 结果
sr_results = []

# 遍历 graphs_json 文件夹中的每个子文件夹
for subfolder in root_path.iterdir():
    if subfolder.is_dir():  # 判断是否是子文件夹
        print(f"Processing subfolder: {subfolder}")
        
        # 统计失败次数（key=协议名, val=失败次数）
        failure_counts = {func.__name__: 0 for func in funcs}
        # 统计所有用户组合的数量
        combination_counter = 0

        # 遍历当前子文件夹中的所有文件
        for file in subfolder.iterdir():
            if file.is_file():  # 判断是否是文件
                print(f"  Processing file: {file}")
                
                class_folder = subfolder.name
                file_path = file.with_suffix(".png")
                file_name = file_path.name
                er_folder_path = DATA_PATHS["output_results"] / class_folder
                er_folder_path.mkdir(exist_ok=True)
                er_topology_folder_path = er_folder_path.joinpath(Path(file_name).stem)
                er_topology_folder_path.mkdir(exist_ok=True)
                print(er_topology_folder_path)
                communities_output_dir = DATA_PATHS["communities"] / class_folder 
                communities_output_dir.mkdir(parents=True, exist_ok=True) 
                communities_output_path = communities_output_dir / file_name
                G, users, pos = load_data(file)
                nx.draw(G, pos, with_labels=True, node_color="lightblue", edge_color="gray")
                G = network(G)
                communities, users_node_combination = multi_iterative_score_partition_with_drawing(
                    G, users[0], alpha, beta, max_rounds, shuffle_nodes, pos, communities_output_path
                )
                
                # 针对每个用户节点组合
                # 抽取随机 100 个组合
                sampled_combinations = np.random.choice(
                    len(users_node_combination),
                    size=min(100, len(users_node_combination)),
                    replace=False
                    )
                for sampled_idx, original_idx in enumerate(sampled_combinations):
                    combo = users_node_combination[original_idx]
                # for combo_idx,combo in enumerate(users_node_combination):
                #     combination_counter += 1
                    print(f'The combination is {combo}')


                    # =========== 新增：组合详细信息 ===========
                    combination_sr = {
                        "Combination_ID": f"combo_{sampled_idx}",
                        "Nodes": str(combo),  # 将节点列表转为字符串，如 "[2,5,9]"
                    }

                    ER = np.zeros((len(funcs), len(p_range)))
                    # for i,p in enumerate(tqdm.tqdm(p_range)):# tqdm_notebook 
                    #     set_p_edge(G,p_op = p)
                    #     for j,function in enumerate(funcs):         
                    #         er,multipartite_gen_time, links_used = function(G,combo,timesteps=timesteps,reps=reps)
                    #         ER[j,i]+=er
                    results = Parallel(n_jobs=-1, verbose=10)(
                        delayed(process_single_p)(G, combo, p, funcs, timesteps, reps)
                        for p in p_range
                    )

                    # 将结果填充到 ER 矩阵
                    for i, p_ers in enumerate(results):
                        ER[:, i] = p_ers

                    plot_er_vs_p(p_range, ER, funcs, cols, er_topology_folder_path.joinpath(f'result_for_{str(combo)}'))
                    # 对每个协议统计有多少个 p 的 ER==0
                    # 计算每个协议的SR（针对当前组合）

                        # =========== 修改：计算成功率比例 ===========
                    for func_idx, func in enumerate(funcs):
                        protocol_er = ER[func_idx, :]
                        zero_count = np.sum(protocol_er < 1e-10)
                        success_ratio = 1 - (zero_count / len(p_range))  # 计算成功率比例
                        combination_sr[func.__name__] = round(success_ratio, 3)  # 保留3位小数

                        # =========== 写入CSV文件 ===========
                    output_subfolder_csv_path = er_topology_folder_path.joinpath(f"{file.stem}_sr_details.csv")

                    # 定义字段（首次运行时写入表头）
                    fieldnames = ["Combination_ID", "Nodes"] + [func.__name__ for func in funcs]

                    # 判断是否需要写表头
                    write_header = not output_subfolder_csv_path.exists()

                    with open(output_subfolder_csv_path, mode="a", newline="") as subfile:
                        csv_writer = csv.DictWriter(subfile, fieldnames=fieldnames)
                        if write_header:
                            csv_writer.writeheader()
                        csv_writer.writerow(combination_sr)

                    # for idx, func in enumerate(funcs):
                    #     protocol_er = ER[idx, :]
                    #     # 如果浮点运算中会出现极小值，可以改成: 
                    #     #zero_count = np.sum(protocol_er < 1e-10)
                    #     zero_count = np.sum(protocol_er == 0)
                        
                    #     # 如果出现 0 的次数超过 zero_threshold，就算“失败”
                    #     if zero_count > zero_threshold:
                    #         failure_counts[func.__name__] += 1
                    # print(f"The failure count is {failure_counts}")

#         # =========== 计算每个协议的 SR ===========
#         sr_for_protocols = {}
#         for protocol_name, failures in failure_counts.items():
#             if combination_counter > 0:
#                 sr_for_protocols[protocol_name] = (combination_counter - failures) / combination_counter
#             else:
#                 sr_for_protocols[protocol_name] = 0.0
        
#         print(f"Subfolder: {subfolder}, SR: {sr_for_protocols}")
        
#         # 保存结果到列表
#         sr_for_protocols["Subfolder"] = subfolder.name
#         sr_results.append(sr_for_protocols)

#         # =========== 保存每个子文件夹的单独 CSV ===========  
#         output_subfolder_csv_path = DATA_PATHS["output_results"].joinpath(f"{subfolder.name}_sr_results.csv")
#         with open(output_subfolder_csv_path, mode="w", newline="") as subfile:
#             fieldnames = ["Subfolder"] + [func.__name__ for func in funcs]
#             csv_writer = csv.DictWriter(subfile, fieldnames=fieldnames)
#             csv_writer.writeheader()
#             csv_writer.writerow(sr_for_protocols)
        
#         print(f"子文件夹 {subfolder.name} 的 SR 结果已保存到 {output_subfolder_csv_path}")

# # =========== 最后保存总的 CSV ===========  
# output_csv_path = DATA_PATHS["output_results"].joinpath("protocol_sr_results.csv")
# with open(output_csv_path, mode="w", newline="") as csvfile:
#     fieldnames = ["Subfolder"] + [func.__name__ for func in funcs]
#     csv_writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
#     csv_writer.writeheader()
#     csv_writer.writerows(sr_results)

# print(f"总的协议 SR 结果已保存到 {output_csv_path}")


Processing subfolder: E:\quantum_testing\quantum_repeaters\graphs_json\class_0
  Processing file: E:\quantum_testing\quantum_repeaters\graphs_json\class_0\TOP_104_USA100.json
E:\quantum_testing\quantum_repeaters\new_result\class_0\TOP_104_USA100
First node ID: 39, Degree: 6
The combination is [39, 14, 32, 35, 83, 100, 52]


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    7.7s
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:   20.4s
[Parallel(n_jobs=-1)]: Done  25 out of  50 | elapsed:   44.5s remaining:   44.5s
[Parallel(n_jobs=-1)]: Done  31 out of  50 | elapsed:  1.0min remaining:   37.2s
[Parallel(n_jobs=-1)]: Done  37 out of  50 | elapsed:  1.3min remaining:   27.1s
[Parallel(n_jobs=-1)]: Done  43 out of  50 | elapsed:  1.5min remaining:   14.4s
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:  1.6min finished


[[1.00000000e+00 1.00000000e+00 1.00000000e+00 9.95024876e-01
  9.95024876e-01 9.95024876e-01 9.80392157e-01 9.52380952e-01
  9.61538462e-01 9.38967136e-01 9.04977376e-01 8.69565217e-01
  8.43881857e-01 7.75193798e-01 7.54716981e-01 6.80272109e-01
  6.41025641e-01 5.39083558e-01 5.27704485e-01 3.85356455e-01
  3.57781753e-01 3.01659125e-01 2.59403372e-01 2.27014756e-01
  1.52439024e-01 1.17370892e-01 7.80640125e-02 5.07731959e-02
  3.66106813e-02 2.25790266e-02 1.55705996e-02 7.88483796e-03
  4.35823461e-03 3.31714252e-03 1.11394017e-03 4.07352717e-04
  1.51103052e-04 5.01228009e-05 5.00050005e-05 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [1.00000000e+00 9.09090909e-01 8.33333333e-01 7.84313725e-01
  6.51465798e-01 5.30503979e-01 5.24934383e-01 4.44444444e-01
  3.75939850e-01 3.09119011e-01 2.90697674e-01 2.30946882e-01
  2.07468880e-01 1.34408602e-01 1.206

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    8.6s
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:   24.8s
[Parallel(n_jobs=-1)]: Done  25 out of  50 | elapsed:   44.8s remaining:   44.8s
[Parallel(n_jobs=-1)]: Done  31 out of  50 | elapsed:  1.0min remaining:   37.5s
[Parallel(n_jobs=-1)]: Done  37 out of  50 | elapsed:  1.3min remaining:   28.1s
[Parallel(n_jobs=-1)]: Done  43 out of  50 | elapsed:  1.5min remaining:   14.2s
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:  1.6min finished


[[1.00000000e+00 1.00000000e+00 9.95024876e-01 1.00000000e+00
  1.00000000e+00 9.75609756e-01 9.95024876e-01 9.80392157e-01
  9.47867299e-01 9.38967136e-01 9.38967136e-01 9.21658986e-01
  8.33333333e-01 8.29875519e-01 7.75193798e-01 7.35294118e-01
  6.45161290e-01 5.88235294e-01 5.24934383e-01 4.21052632e-01
  3.90625000e-01 2.67379679e-01 2.21729490e-01 1.60771704e-01
  1.26103405e-01 9.10746812e-02 6.89893067e-02 4.20079815e-02
  2.78104768e-02 1.69297881e-02 8.08956302e-03 4.63727608e-03
  2.56454095e-03 1.52182184e-03 6.73505336e-04 3.05343511e-04
  5.00901623e-05 1.00547987e-04 1.00265704e-04 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [1.00000000e+00 8.77192982e-01 8.40336134e-01 7.24637681e-01
  6.49350649e-01 5.20833333e-01 4.62962963e-01 3.92156863e-01
  3.51493849e-01 2.96735905e-01 2.38379023e-01 1.83992640e-01
  1.54440154e-01 1.11234705e-01 8.924

...

[[1.00000000e+00 1.00000000e+00 1.00000000e+00 9.90099010e-01
  9.61538462e-01 9.47867299e-01 9.21658986e-01 8.92857143e-01
  8.73362445e-01 8.77192982e-01 8.54700855e-01 7.51879699e-01
  7.81250000e-01 6.77966102e-01 6.21118012e-01 6.38977636e-01
  6.17283951e-01 5.61797753e-01 4.97512438e-01 5.12820513e-01
  3.71057514e-01 3.65630713e-01 3.44234079e-01 2.81293952e-01
  2.64200793e-01 2.27272727e-01 2.33372229e-01 1.81983621e-01
  1.69204738e-01 1.67364017e-01 1.31406045e-01 1.00806452e-01
  8.54700855e-02 7.13521227e-02 6.29524709e-02 5.77984316e-02
  4.98877526e-02 3.60710322e-02 2.79861512e-02 2.40634755e-02
  1.90147893e-02 1.33014099e-02 1.13607675e-02 9.34293154e-03
  6.52680653e-03 4.11149404e-03 3.22845738e-03 1.53789044e-03
  2.14757709e-03 6.22826595e-04]
 [1.00000000e+00 9.38967136e-01 8.62068966e-01 8.00000000e-01
  7.93650794e-01 6.77966102e-01 6.21118012e-01 5.58659218e-01
  5.44959128e-01 4.65116279e-01 4.42477876e-01 3.77358491e-01
  3.78071834e-01 2.90697674e-01 2.574

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    0.5s
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:    1.4s
[Parallel(n_jobs=-1)]: Done  25 out of  50 | elapsed:    3.0s remaining:    3.0s
[Parallel(n_jobs=-1)]: Done  31 out of  50 | elapsed:    5.4s remaining:    3.2s
[Parallel(n_jobs=-1)]: Done  37 out of  50 | elapsed:    8.0s remaining:    2.7s
[Parallel(n_jobs=-1)]: Done  43 out of  50 | elapsed:   10.0s remaining:    1.5s
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:   12.5s finished


[[1.00000000e+00 1.00000000e+00 9.90099010e-01 9.80392157e-01
  9.70873786e-01 9.47867299e-01 9.25925926e-01 8.88888889e-01
  8.43881857e-01 8.54700855e-01 8.13008130e-01 7.49063670e-01
  7.27272727e-01 6.89655172e-01 6.55737705e-01 5.81395349e-01
  5.39083558e-01 5.54016620e-01 4.61893764e-01 4.37636761e-01
  3.87596899e-01 3.75234522e-01 3.20000000e-01 2.98953662e-01
  2.46002460e-01 2.23964166e-01 1.80995475e-01 1.86219739e-01
  1.66805671e-01 1.40745954e-01 1.18553646e-01 1.15406809e-01
  9.12825194e-02 7.06214689e-02 6.52954620e-02 4.68378507e-02
  4.57471264e-02 3.48754448e-02 2.72187636e-02 2.33187323e-02
  1.57842158e-02 1.29515419e-02 9.44192830e-03 9.64704081e-03
  7.16948666e-03 4.52062535e-03 3.10970343e-03 1.82451484e-03
  1.11081724e-03 1.38364110e-03]
 [1.00000000e+00 8.73362445e-01 7.87401575e-01 7.11743772e-01
  6.02409639e-01 5.57103064e-01 4.58715596e-01 4.53514739e-01
  3.63636364e-01 3.14960630e-01 2.86944046e-01 2.81293952e-01
  2.38379023e-01 1.83654729e-01 1.629

C:\Users\Ao\AppData\Local\Temp\ipykernel_1680\581972065.py:33: SyntaxWarning: invalid escape sequence '\m'
  plt.ylabel('ER ($\mathregular{GHZ}_5/\ \\mathregular{T_{slot}}$)',fontsize=fontsize)


MemoryError: 